In [3]:
#-------------------------------------------------------------------------------
# elftools example: dwarf_lineprogram_filenames.py
#
# In the .debug_line section, the Dwarf line program generates a matrix
# of address-source references. This example demonstrates accessing the state
# of each line program entry to retrieve the underlying filenames.
#
# William Woodruff (william@yossarian.net)
# This code is in the public domain
#-------------------------------------------------------------------------------
from __future__ import print_function
from collections import defaultdict
import os
import sys
import posixpath

# If pyelftools is not installed, the example can also run from the root or
# examples/ dir of the source distribution.
# sys.path[0:0] = ['.', '..']

from elftools.elf.elffile import ELFFile


def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        dwarfinfo = elffile.get_dwarf_info()
        
        with open('lineprogram.s', 'w') as outFile:
            for CU in dwarfinfo.iter_CUs():
                print('  Found a compile unit at offset %s, length %s' % (
                    CU.cu_offset, CU['unit_length']))

                # Every compilation unit in the DWARF information may or may not
                # have a corresponding line program in .debug_line.
                line_program = dwarfinfo.line_program_for_CU(CU)
                if line_program is None:
                    print('  DWARF info is missing a line program for this CU')
                    continue

                
                
                # Print a reverse mapping of filename -> #entries
                filename_map= line_entry_mapping(line_program)
                if(len(list(filename_map.items())))>1:
                    print(list(filename_map.items()))

                CU_DIR_PATH = None
                CU_FILENAME = None
                for attr in CU.get_top_DIE().attributes.values():
                    if attr.name == 'DW_AT_name':
                        CU_FILENAME = attr.value.decode("utf-8")
                    if attr.name == 'DW_AT_comp_dir':
                        CU_DIR_PATH = attr.value.decode("utf-8")
#                 print(CU_FILENAME)
                
                for le in line_program.get_entries():
                    if le.state is not None:
                        
                        if True or str(hex(le.state.address)) == '0x2469b':
                            print(le)
                        src_filename = lpe_filename (line_program, le.state.file)
            
                        outFile.write(str(hex(le.state.address))+"    "+str(le.state.line)+"  "+    str(le.state.column)+"   "+os.path.join(CU_DIR_PATH,CU_FILENAME)+"   "+ src_filename   +"\n")

    
    #                         if src_filename==CU_FILENAME:
# #                         print(le)
# #                         print( str(le.state.address)+"    "+str(le.state.line)+"  "+    str(le.state.column)+"\n")
#                             outFile.write(str(hex(le.state.address))+"    "+str(le.state.line)+"  "+    str(le.state.column)+"   "+os.path.join(CU_DIR_PATH,CU_FILENAME)+"   "+ src_filename   +"\n")
#                         else:
#                             outFile.write(str(hex(le.state.address))+"    "+str(le.state.line)+"  "+    str(le.state.column)+"   "+os.path.join(CU_DIR_PATH,CU_FILENAME)+"   "+ src_filename   +"\n")
                            
def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename (line_program, lpe.state.file)
        filename_map[filename] += 1

#     for filename, lpe_count in filename_map.items():
#         print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index):

    lp_header = line_program.header
    file_entries = lp_header["file_entry"]

    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()

    directory = lp_header["include_directory"][dir_index - 1]
    return posixpath.join(directory, file_entry.name).decode()



# filePath = '/home/nahid/reverse/binaries/c_many/stacktest'
filePath =  './../../binaries/gnuit/src/gitfm'
process_file(filePath)


Processing file: ./../../binaries/gnuit/src/gitfm
  Found a compile unit at offset 0, length 10410
LineProgramEntry(command=1, is_extended=False, args=[], state=<LineState 7f4a7ca17fd0:
  address = 0x4fd9
  file = 1
  line = 348
  column = 1
  is_stmt = 1
  basic_block = False
  end_sequence = False
  prologue_end = False
  epilogue_begin = False
  isa = 0
  discriminator = 0>
)
LineProgramEntry(command=131, is_extended=False, args=[1, 8, 0], state=<LineState 7f4a7ca10040:
  address = 0x4fe1
  file = 1
  line = 349
  column = 19
  is_stmt = 1
  basic_block = False
  end_sequence = False
  prologue_end = False
  epilogue_begin = False
  isa = 0
  discriminator = 0>
)
LineProgramEntry(command=102, is_extended=False, args=[0, 6, 0], state=<LineState 7f4a7ca10070:
  address = 0x4fe7
  file = 1
  line = 349
  column = 8
  is_stmt = 1
  basic_block = False
  end_sequence = False
  prologue_end = False
  epilogue_begin = False
  isa = 0
  discriminator = 0>
)
LineProgramEntry(command=90, is_e

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  Found a compile unit at offset 73283, length 2802
LineProgramEntry(command=1, is_extended=False, args=[], state=<LineState 7f4a7c9f9b50:
  address = 0x24c40
  file = 1
  line = 62
  column = 1
  is_stmt = 1
  basic_block = False
  end_sequence = False
  prologue_end = False
  epilogue_begin = False
  isa = 0
  discriminator = 0>
)
LineProgramEntry(command=103, is_extended=False, args=[1, 6, 0], state=<LineState 7f4a7c9f9a30:
  address = 0x24c46
  file = 1
  line = 63
  column = 3
  is_stmt = 1
  basic_block = False
  end_sequence = False
  prologue_end = False
  epilogue_begin = False
  isa = 0
  discriminator = 0>
)
LineProgramEntry(command=17, is_extended=False, args=[-1, 0, 0], state=<LineState 7f4a7c9f9610:
  address = 0x24c46
  file = 1
  line = 62
  column = 1
  is_stmt = False
  basic_block = False
  end_sequence = False
  prologue_end = False
  epilogue_begin = False
  isa = 0
  discriminator = 0>
)
LineProgramEntry(command=75, is_extended=False, args=[1, 4, 0], state=<LineSt